In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, monotonically_increasing_id, when, first, regexp_replace, split
from pyspark.sql.window import Window
import os

# 1. Tworzymy sesję Spark
spark = SparkSession.builder.appName("FASTQ_Clean_Header").getOrCreate()

# 2. Ścieżka do katalogu z FASTQ
fastq_dir = "/Volumes/databrics_2/default/fastq"

# 3. Lista plików FASTQ
files = ["SRR16356246_1.fastq", "SRR16356246_2.fastq"]

for file_name in files:
    file_path = os.path.join(fastq_dir, file_name)
    
    # 4. Wczytanie pliku do DataFrame Spark
    lines_df = spark.read.text(file_path)
    
    # 5. Dodanie numeru linii
    window = Window.orderBy(monotonically_increasing_id())
    lines_with_id = lines_df.withColumn("line_number", row_number().over(window) - 1)
    
    # 6. Określenie typu linii FASTQ
    lines_typed = lines_with_id.withColumn(
        "line_type",
        when(col("line_number") % 4 == 0, "header")
        .when(col("line_number") % 4 == 1, "sequence")
        .when(col("line_number") % 4 == 2, "separator")
        .when(col("line_number") % 4 == 3, "quality")
    )
    
    # 7. Utworzenie record_id
    lines_with_record = lines_typed.withColumn(
        "record_id",
        (col("line_number") / 4).cast("integer")
    )
    
    # 8. Pivot: format szeroki (1 wiersz na odczyt)
    fastq_wide = lines_with_record.groupBy("record_id").pivot("line_type").agg(
        first("value").alias("value")
    )

    # 9. Kolumny pivotowane mają teraz aliasy 'value' - nadajemy właściwe nazwy
    fastq_wide = fastq_wide.select(
        col("record_id"),
        col("header").alias("header") if "header" in fastq_wide.columns else col("value").alias("header"),
        col("sequence").alias("sequence") if "sequence" in fastq_wide.columns else col("value").alias("sequence"),
        col("quality").alias("quality") if "quality" in fastq_wide.columns else col("value").alias("quality")
    )
    
    # 10. Czyszczenie nagłówka, wyodrębnienie read_id
    fastq_clean = fastq_wide.withColumn(
        "read_id",
        split(regexp_replace(col("header"), "^@", ""), " ")[0]
    ).select("record_id", "read_id", "sequence", "quality")
    
    # 11. Wyświetlenie pierwszych 5 rekordów
    print(f"\nPlik: {file_name} - pierwsze 5 rekordów po czyszczeniu nagłówka:")
    fastq_clean.show(5, truncate=False)
    
    # 12. Liczba odczytów w pliku
    total_reads = fastq_clean.count()
    print(f"Liczba odczytów w pliku {file_name}: {total_reads}\n")


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(



Plik: SRR16356246_1.fastq - pierwsze 5 rekordów po czyszczeniu nagłówka:


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
